# AutoGen Test Run

Source: https://microsoft.github.io/autogen/docs/topics/task_decomposition/#approach-2-group-chat

## ENV and dependencies

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from typing_extensions import Annotated

from autogen import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    config_list_from_json,
    register_function
)
from autogen.agentchat.contrib import agent_builder
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor

/home/xiaolishen/projects/FinAgents/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## LLM Configuration

In [3]:
config_gpt_4o = {
    "model": os.environ.get("GPT_4O_MODEL"),
    "api_type": "azure",
    "api_key": os.environ.get("GPT_4O_API_KEY"),
    "base_url": os.environ.get("GPT_4O_AZURE_ENDPOINT"),
    "api_version": os.environ.get("GPT_4O_API_VERSION"),
}


config_gpt_4_turbo = {
    "model": os.environ.get("GPT_4_TURBO_MODEL"),
    "api_type": "azure",
    "api_key": os.environ.get("GPT_4_TURBO_API_KEY"),
    "base_url": os.environ.get("GPT_4_TURBO_AZURE_ENDPOINT"),
    "api_version": os.environ.get("GPT_4_TURBO_API_VERSION"),
}


config_gpt_35_turbo = {
    "model": os.environ.get("GPT_35_TURBO_MODEL"),
    "api_type": "azure",
    "api_key": os.environ.get("GPT_35_TURBO_API_KEY"),
    "base_url": os.environ.get("GPT_35_TURBO_AZURE_ENDPOINT"),
    "api_version": os.environ.get("GPT_35_TURBO_API_VERSION"),
}

## Scaffolding Test: Single Chatbot Agent

In [4]:
# Test out simplest chatbot agent
from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [config_gpt_35_turbo]},
    code_execution_config=False,
    function_map=None,
    human_input_mode="NEVER",
)

In [5]:
reply = agent.generate_reply(messages=[{"content": "What do you know about finance?", "role": "user"}])
print(reply)

I can help provide information on a variety of finance topics, such as personal finance, investments, budgeting, saving, credit management, and more. I can also assist with explaining financial concepts, offering tips and strategies for financial planning, and answering specific questions you might have about finance. Let me know how I can assist you further!


## Multi-Agent Test: Group Chat Task Decomposition

https://microsoft.github.io/autogen/docs/topics/task_decomposition#approach-2-group-chat

In [6]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Give the task, and send instructions to writer to refine the blog post.",
    code_execution_config=False,
)

planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Given a task, please determine what information is needed to complete the task.
Please note that the information will all be retrieved using Python code. Please only suggest information that can be retrieved using Python code.
""",
    llm_config={"config_list": [config_gpt_35_turbo]},
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config={"config_list": [config_gpt_4_turbo], "cache_seed": None},
    system_message="""Engineer. You write python/bash to retrieve relevant information. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor. 
    Always provide bash code block to install dependency first.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
Write code to print the output so that the others can access the result. When you use code to plot data, save the graph to an image file in the work dir and print out the file path. 
""",
)

writer = AssistantAgent(
    name="Writer",
    llm_config={"config_list": [config_gpt_4_turbo], "cache_seed": None},
    system_message="""Writer. Please write blogs in markdown format (with relevant titles) and put the content in pseudo ```md``` code block. You will write it for a task based on previous chat history. Don't write any code.""",
)

os.makedirs("paper", exist_ok=True)
code_executor = UserProxyAgent(
    name = "Executor",
    system_message = "Executor. Execute the code written by the engineer and report the result.",
    description = "Executor should always be called after the engineer has written the code to be executed.",
    human_input_mode = "ALWAYS",
    code_execution_config = {
        "last_n_messages": 3,
        "executor": LocalCommandLineCodeExecutor(work_dir="paper"),
    },
)

In [19]:
from typing import List, Any, Tuple
def reply_func(
    recipient: ConversableAgent,
    messages: Optional[List[Dict]] = None,
    sender: Optional[Agent] = None,
    config: Optional[Any] = None,
) -> Tuple[bool, Union[str, Dict, None]]:
    print(f"recipient: {recipient}")
    print(f"messages: {messages}")
    print(f"sender: {sender}")
    print(f"config: {config}")

In [24]:
user_proxy.register_reply(planner, reply_func)

In [27]:
groupchat = GroupChat(
    agents = [user_proxy, planner, engineer, writer, code_executor],
    messages = [],
    max_round = 10,
    speaker_selection_method = "auto",
)

manager = GroupChatManager(
    groupchat=groupchat, 
    llm_config={
        "config_list": [config_gpt_4_turbo], 
        "cache_seed": None
    }
)

In [28]:
user_proxy.initiate_chat(manager, message="Say hello to me")

Admin (to chat_manager):

Say hello to me

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

To say hello in Python, I would need to know if you want the Python code to print "hello" to the console or display it in some other way.

--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

To say "hello" in Python, you can use the `print` function to display the message in the console. Here's a simple Python script that prints "hello":

```python
print("Hello!")
```

You can run this code in any Python environment, and it will display "Hello!" in the console.

--------------------------------------------------------------------------------

Next speaker: Executor


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
Executor (to chat_manager):

exitcode: 

ChatResult(chat_id=None, chat_history=[{'content': 'Say hello to me', 'role': 'assistant'}, {'content': 'To say hello in Python, I would need to know if you want the Python code to print "hello" to the console or display it in some other way.', 'name': 'Planner', 'role': 'user'}, {'content': 'To say "hello" in Python, you can use the `print` function to display the message in the console. Here\'s a simple Python script that prints "hello":\n\n```python\nprint("Hello!")\n```\n\nYou can run this code in any Python environment, and it will display "Hello!" in the console.', 'name': 'Engineer', 'role': 'user'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output: Hello!\n', 'name': 'Executor', 'role': 'user'}, {'content': '```md\n# How to Say "Hello" in Python\n\nPython, renowned for its simplicity and readability, provides a straightforward way to start your programming journey. One of the initial steps in learning any programming language is understanding how to display text. In 

In [8]:
task = (
    f"Today is {datetime.now().date()}. Write a blogpost about the stock price performance of Nvidia in the past month."
)
print(task)

Today is 2024-06-22. Write a blogpost about the stock price performance of Nvidia in the past month.


In [9]:
with Cache.disk(cache_seed=41) as cache:
    chat_history = user_proxy.initiate_chat(
        manager,
        message=task,
        cache=cache,
    )

Admin (to chat_manager):

Today is 2024-06-22. Write a blogpost about the stock price performance of Nvidia in the past month.

--------------------------------------------------------------------------------



Next speaker: Planner

Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia in the past month, we would need the following information retrieved via Python code:

1. Historical stock price data for Nvidia for the past month.
2. Specific date range within the past month for analysis.
3. Opening and closing prices of Nvidia stock for each trading day in the past month.
4. Highest and lowest prices of Nvidia stock in the past month.
5. Any significant events or news related to Nvidia that could have impacted its stock price in the past month.
6. Calculations or analysis on the percent change in stock price over the past month.
7. Visualization of Nvidia stock price performance in the past month (optional).

--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

To retrieve the stock price data required for writing the blog post about Nvidia's performance in the past mon

## Preview generated blog article


\============
### Nvidia's Stock Price Soars: An Analysis of the Past Month's Performance

Nvidia, renowned for its advancements in GPUs and AI technologies, has seen an impressive surge in its stock price over the last month. Let's delve into the details of this upward trend.

#### Stock Performance Overview
From May 7, 2024, through June 7, 2024, Nvidia's stock has shown remarkable growth:

- **Starting Price:** $905.54 on May 7, 2024
- **Ending Price:** $1209.98 on June 6, 2024
- **Peak Price:** $1224.40 on June 5, 2024

The stock experienced its most significant jump on May 23, 2024, where the closing price shot up by roughly 9% in a single day.

![Nvidia Stock Price May-June 2024](paper/Nvidia_Stock_Price_May_June_2024.png)

#### Statistical Insights
Over this period, the average closing price was approximately $1019.06, indicating substantial growth and investor confidence.

- **Volatility:** The standard deviation of the closing prices was around $115.48, suggesting a higher level of risk that matched the potential for greater returns.

#### Contributing Factors
A blend of exceptional quarterly earnings, innovative product launches, and strategic partnerships seem to underpin this bullish trend.

#### Looking Ahead
Given the current data and market trends, Nvidia's outlook appears positive, though it's crucial for investors to stay informed about both market and technological shifts that could impact future stock performance.

Stay updated with real-time analysis and more at [Your Financial Blog]. *Note: Assume financial data may vary slightly and check real-time data for the most current information.*


## Inspect

In [10]:
groupchat

GroupChat(agents=[<autogen.agentchat.user_proxy_agent.UserProxyAgent object at 0x7f8b24a45bb0>, <autogen.agentchat.assistant_agent.AssistantAgent object at 0x7f8b24a461e0>, <autogen.agentchat.assistant_agent.AssistantAgent object at 0x7f8b24a45f10>, <autogen.agentchat.assistant_agent.AssistantAgent object at 0x7f8b24a47a40>, <autogen.agentchat.user_proxy_agent.UserProxyAgent object at 0x7f8b24a670b0>], messages=[{'content': 'Today is 2024-06-22. Write a blogpost about the stock price performance of Nvidia in the past month.', 'role': 'user', 'name': 'Admin'}, {'content': 'To write a blog post about the stock price performance of Nvidia in the past month, we would need the following information retrieved via Python code:\n\n1. Historical stock price data for Nvidia for the past month.\n2. Specific date range within the past month for analysis.\n3. Opening and closing prices of Nvidia stock for each trading day in the past month.\n4. Highest and lowest prices of Nvidia stock in the past 

In [11]:
groupchat.messages

[{'content': 'Today is 2024-06-22. Write a blogpost about the stock price performance of Nvidia in the past month.',
  'role': 'user',
  'name': 'Admin'},
 {'content': 'To write a blog post about the stock price performance of Nvidia in the past month, we would need the following information retrieved via Python code:\n\n1. Historical stock price data for Nvidia for the past month.\n2. Specific date range within the past month for analysis.\n3. Opening and closing prices of Nvidia stock for each trading day in the past month.\n4. Highest and lowest prices of Nvidia stock in the past month.\n5. Any significant events or news related to Nvidia that could have impacted its stock price in the past month.\n6. Calculations or analysis on the percent change in stock price over the past month.\n7. Visualization of Nvidia stock price performance in the past month (optional).',
  'role': 'user',
  'name': 'Planner'},
 {'content': 'To retrieve the stock price data required for writing the blog po

In [12]:
groupchat.agents

In [16]:
planner.chat_messages

defaultdict(list,
            {<autogen.agentchat.groupchat.GroupChatManager at 0x7f8b25836c60>: [{'content': 'Today is 2024-06-22. Write a blogpost about the stock price performance of Nvidia in the past month.',
               'name': 'Admin',
               'role': 'user'},
              {'content': 'To write a blog post about the stock price performance of Nvidia in the past month, we would need the following information retrieved via Python code:\n\n1. Historical stock price data for Nvidia for the past month.\n2. Specific date range within the past month for analysis.\n3. Opening and closing prices of Nvidia stock for each trading day in the past month.\n4. Highest and lowest prices of Nvidia stock in the past month.\n5. Any significant events or news related to Nvidia that could have impacted its stock price in the past month.\n6. Calculations or analysis on the percent change in stock price over the past month.\n7. Visualization of Nvidia stock price performance in the past mon

In [17]:
planner.system_message

'Planner. Given a task, please determine what information is needed to complete the task.\nPlease note that the information will all be retrieved using Python code. Please only suggest information that can be retrieved using Python code.\n'

## AutoGen with Assistant API

## AutoGen with RAG